In [1]:
#TODO: Everything
using LinearAlgebra



In [2]:
""" 
Podatkovni tip za pasovno matriko.
Hrani vektor za diagonalo, vektor vektorjev za neničelne vrstice pod diagonalo in vektor vektorjev za neničelne vrstice nad diagonalo

"""

struct PasovnaMatrika{T} <: AbstractArray{T,2}
        d::Vector{T}  
        s::Vector{Vector{T}}
        z::Vector{Vector{T}}
end

import Base:size,getindex,setindex!

"""
    size(A)

    Vrne velikost pasovne matrike

"""

function size(A::PasovnaMatrika)
    return (size(A.d)[1],size(A.d)[1])
end

"""
    getindex(A,I) ali A[i,j]

    Vrne element na mestu i,j

"""

function getindex(A::PasovnaMatrika, I::Vararg{Int,2})
    
    if I > size(A)
        return 0
    end
    
    i =  I[1] - I[2]

    if i == 0
        return A.d[I[1]]
    end
    if i > 0 && i <= size(A.s)[1]
        return A.s[i][I[1]-i]
    end
    i = -i
    if i > 0 && i <= size(A.z)[1]
        return A.z[i][I[2]-i]
    end
    return 0

end

"""
    setindex(A,v,I) ali A[i,j] = v

    Nastavi element na mestu i, j na v, ce je to mesto v pasu, drugače ne naredi ničesar.

"""

function setindex!(A::PasovnaMatrika, v, I::Vararg{Int,2})
    
    if I > size(A)
        return
    end
    
    i =  I[1] - I[2]

    if i == 0
        A.d[I[1]] = v
    end
    if i > 0 && i <= size(A.s)[1]
        A.s[i][I[1]-i] = v
    end
    i = -i
    if i > 0 && i <= size(A.z)[1]
        A.z[i][I[2]-i] = v
    end
end

setindex! (generic function with 83 methods)

In [144]:
import Base:*, \
import LinearAlgebra:lu

function *(A::PasovnaMatrika, v::Vector)
    y = zeros(size(A.d))
    
    for i = 1 : size(A.d)[1]
       y[i] = v[i] * A.d[i]
    end
    
    for i = 1 : size(A.s)[1]
       for j = 1 : size(A.s[i])[1]
            y[j+i] += v[j] * A.s[i][j]
        end
    end
    
    for i = 1 : size(A.z)[1]
       for j = 1 : size(A.z[i])[1]
            y[j] += v[j+i] * A.z[i][j]
        end
    end
    
    return y
end

function lu(A::PasovnaMatrika)
    
    n = length(A.d)
    ns = length(A.s)
    nz = length(A.z)
    
    U = PasovnaMatrika(deepcopy(A.d),deepcopy(A.s),deepcopy(A.z))
    #U = copy(A)
    L = SpodnjePasovnaMatrika(ones(length(A.d)),deepcopy(A.s))

    for i = 1:n-1
        for j = i+1:i+ns
            if j > n
                break
            end
            l = U[j,i]/U[i,i]
            L[j,i] = l
            U[j,:] -= U[i,:]*l
        end
    end

    #display(U)
    
    U = ZgornjePasovnaMatrika(U.d,U.z)
    
    return L, U
end

function \(A::PasovnaMatrika, b::Vector)
     
    x = copy(b)
    n = length(b)
    ns = length(A.s)
    nz = length(A.z)

    for i = 1:n-1
        for j = i+1:i+ns
            if j > n
                break
            end
            #row = A[i,:]
            l = A[j,i]/A[i,i]
            A[j,:] -= A[i,:]*l
            x[j] -= x[i]*l
        end
    end
    
    display(A)
    display(x)
    
    for i = n:-1:1
        temp = x[i]
        for j = i+1:n
            display(j)
            if j > nz+i
                break
            end
            temp -= x[j]*A[i,j]
            display(temp)
        end
        x[i] = temp/A[i,i]
        display((temp,A[i,i]))
    end
    
    return x
end

function \(A::ZgornjePasovnaMatrika, b::Vector)
     
    x = copy(b)
    n = length(b)
    nz = length(A.z)

    for i = n:-1:1
        temp = x[i]
        for j = i+1:n
            if j > nz+i
                break
            end
            temp -= x[j]*A[i,j]
        end
        x[i] = temp/A[i,i]
    end
    
    return x
end

function \(A::SpodnjePasovnaMatrika, b::Vector)
     
    x = copy(b)
    n = length(b)
    ns = length(A.s)

    display(A)
    
    for i = 1:n-1
        for j = i+1:i+ns
            if j > n
                break
            end
            l = A[j,i]/A[i,i]
            A[j,:] -= A[i,:]*l
            x[j] -= x[i]*l
        end
    end
    
    for i = n:-1:1
        x[i] = x[i]/A[i,i]
    end
    
    return x
end

\ (generic function with 155 methods)

In [145]:
mat = PasovnaMatrika([1.0, 5, 9],[[4,8.0],[7.0]],[[2.0,6.0]])
display(mat)
v=[1.0,2.0,3.0]
display(mat\v)
display(lu(mat)[1])
display(lu(mat)[2])

3×3 PasovnaMatrika{Float64}:
 1.0  2.0  0  
 4.0  5.0  6.0
 7.0  8.0  9.0

3×3 PasovnaMatrika{Float64}:
 1.0   2.0   0  
 0.0  -3.0   6.0
 0.0   0.0  -3.0

3-element Array{Float64,1}:
  1.0
 -2.0
  0.0

(0.0, -3.0)

3

...

3

(-0.33333333333333326, 1.0)

3-element Array{Float64,1}:
 -0.33333333333333326
  0.6666666666666666 
 -0.0                

3×3 SpodnjePasovnaMatrika{Float64}:
 1.0   0    0  
 0.0   1.0  0  
 0.0  -0.0  1.0

3×3 ZgornjePasovnaMatrika{Float64}:
 1.0   2.0   0  
 0    -3.0   6.0
 0     0    -3.0

In [112]:
mat = [1.0 2 0;4 5 6;7 8 9]
display(mat)
#lu(mat)
mat\v

3×3 Array{Float64,2}:
 1.0  2.0  0.0
 4.0  5.0  6.0
 7.0  8.0  9.0

3-element Array{Float64,1}:
 -0.3333333333333333    
  0.6666666666666666    
 -1.0570971181733665e-17

In [23]:
s = [1.0,1.0,1.0]
d = [1.0,1.0,1.0,1.0]
z = [2.0,2.0,2.0]

n = 4

for i = 2:n
    l = s[i-1]/d[i-1]
    d[i] -= l*z[i-1]
    display(PasovnaMatrika(d,[s],[z]))
end


4×4 PasovnaMatrika{Float64}:
 1.0   2.0  0    0  
 1.0  -1.0  2.0  0  
 0     1.0  1.0  2.0
 0     0    1.0  1.0

4×4 PasovnaMatrika{Float64}:
 1.0   2.0  0    0  
 1.0  -1.0  2.0  0  
 0     1.0  3.0  2.0
 0     0    1.0  1.0

4×4 PasovnaMatrika{Float64}:
 1.0   2.0  0    0       
 1.0  -1.0  2.0  0       
 0     1.0  3.0  2.0     
 0     0    1.0  0.333333

In [54]:
mat = PasovnaMatrika([1.0,1.0,1.0,1.0],[[1.0,1.0,1.0]],[[2.0,2.0,2.0]])
v = [1.0,1,1,1]

display(mat)

mat\v
lu(mat)

4×4 PasovnaMatrika{Float64}:
 1.0  2.0  0    0  
 1.0  1.0  2.0  0  
 0    1.0  1.0  2.0
 0    0    1.0  1.0

4×4 PasovnaMatrika{Float64}:
 1.0  2.0  0    0  
 1.0  1.0  2.0  0  
 0    1.0  1.0  2.0
 0    0    1.0  1.0

2-element Array{Int32,1}:
 1
 2

2-element Array{Int32,1}:
 2
 3

2-element Array{Int32,1}:
 3
 4

4×4 PasovnaMatrika{Float64}:
 1.0   2.0  0    0       
 0.0  -1.0  2.0  0       
 0     0.0  3.0  2.0     
 0     0    0.0  0.333333

4-element Array{Float64,1}:
 1.0               
 0.0               
 1.0               
 0.6666666666666667

4×4 Array{Float64,2}:
 1.0   2.0  0.0  0.0     
 0.0  -1.0  2.0  0.0     
 0.0   0.0  3.0  2.0     
 0.0   0.0  0.0  0.333333

4×4 SpodnjePasovnaMatrika{Float64}:
 1.0   0    0    0       
 0.0  -1.0  0    0       
 0    -0.0  3.0  0       
 0     0    0.0  0.333333

In [128]:
mat = [1.0 2 0 0 ; 1 1 2 0 ; 0 1 1 2 ; 0 0 1 1]
mat\v

4-element Array{Float64,1}:
  5.0
 -2.0
 -1.0
  2.0

In [79]:
mat = PasovnaMatrika([3.0,2,1,2],[[3.0,6,2],[2.0,1]],[[5.0,3,2]])
v = [2,1,3,1]

mat*v

4-element Array{Float64,1}:
 11.0
 17.0
 15.0
  9.0

In [38]:
mat = [1.0 2 0;4 5 6;7 8 9]
lu(mat)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0       0.0  0.0
 0.142857  1.0  0.0
 0.571429  0.5  1.0
U factor:
3×3 Array{Float64,2}:
 7.0  8.0        9.0    
 0.0  0.857143  -1.28571
 0.0  0.0        1.5    

In [13]:
mat = PasovnaMatrika([1.0, 5, 9],[[4.0,8],[7.0]],[[2.0,6]])
v=[1,2,3]
mat\v

BoundsError: [91mBoundsError: attempt to access 3×3 PasovnaMatrika{Float64} at index [4, Base.Slice(Base.OneTo(3))][39m

In [129]:
mat = [3 5 0 0; 3 2 3 0; 2 6 1 2; 0 1 2 2]
mat * v

4-element Array{Int64,1}:
 11
 17
 15
  9

In [8]:
""" 
Podatkovni tip za zgornje pasovno matriko.
Hrani vektor za diagonalo in vektor vektorjev za neničelne vrstice nad diagonalo

"""

struct ZgornjePasovnaMatrika{T} <: AbstractArray{T,2}
        d::Vector{T}  
        z::Vector{Vector{T}}
end

import Base:size,getindex,setindex!

"""
    size(A)

    Vrne velikost pasovne matrike

"""

function size(A::ZgornjePasovnaMatrika)
    return (size(A.d)[1],size(A.d)[1])
end

"""
    getindex(A,I) ali A[i,j]

    Vrne element na mestu i,j

"""

function getindex(A::ZgornjePasovnaMatrika, I::Vararg{Int,2})
    
    if I > size(A)
        return 0
    end
    
    i =  I[1] - I[2]

    if i == 0
        return A.d[I[1]]
    end
    i = -i
    if i > 0 && i <= size(A.z)[1]
        return A.z[i][I[2]-i]
    end
    return 0

end

function setindex!(A::ZgornjePasovnaMatrika, v, I::Vararg{Int,2})
    
    if I > size(A)
        return
    end
    
    i =  I[1] - I[2]

    if i == 0
        A.d[I[1]] = v
    end
    i = -i
    if i > 0 && i <= size(A.z)[1]
        A.z[i][I[2]-i] = v
    end
end

setindex! (generic function with 84 methods)

In [9]:
""" 
Podatkovni tip za spodnje pasovno matriko.
Hrani vektor za diagonalo in vektor vektorjev za neničelne vrstice pod diagonalo

"""

struct SpodnjePasovnaMatrika{T} <: AbstractArray{T,2}
        d::Vector{T}  
        s::Vector{Vector{T}}
end

import Base:size,getindex,setindex!

"""
    size(A)

    Vrne velikost pasovne matrike

"""

function size(A::SpodnjePasovnaMatrika)
    return (size(A.d)[1],size(A.d)[1])
end

"""
    getindex(A,I) ali A[i,j]

    Vrne element na mestu i,j

"""

function getindex(A::SpodnjePasovnaMatrika, I::Vararg{Int,2})
    
    if I > size(A)
        return 0
    end
    
    i =  I[1] - I[2]

    if i == 0
        return A.d[I[1]]
    end
    if i > 0 && i <= size(A.s)[1]
        return A.s[i][I[1]-i]
    end
    return 0

end

function setindex!(A::SpodnjePasovnaMatrika, v, I::Vararg{Int,2})
    
    if I > size(A)
        return
    end
    
    i =  I[1] - I[2]

    if i == 0
        A.d[I[1]] = v
    end
    if i > 0 && i <= size(A.s)[1]
        A.s[i][I[1]-i] = v
    end
end

setindex! (generic function with 85 methods)

In [35]:
mat = PasovnaMatrika([1,2,1,1],[[2,2,2],[3,5]],[[2,2,2],[3,5]])

4×4 PasovnaMatrika{Int32}:
 1  2  3  0
 2  2  2  5
 3  2  1  2
 0  5  2  1

In [36]:
mat[1,3] = 1
mat

4×4 PasovnaMatrika{Int32}:
 1  2  1  0
 2  2  2  5
 3  2  1  2
 0  5  2  1

In [40]:
mat = ZgornjePasovnaMatrika([1,2,1,1],[[2,2,2],[3,5]])

4×4 ZgornjePasovnaMatrika{Int64}:
 1  2  3  0
 0  2  2  5
 0  0  1  2
 0  0  0  1

In [41]:
mat = SpodnjePasovnaMatrika([1,2,1,1],[[2,2,2],[3,5]])

4×4 SpodnjePasovnaMatrika{Int64}:
 1  0  0  0
 2  2  0  0
 3  2  1  0
 0  5  2  1

In [10]:
mat = Array([1 1 1 ; 2 3 2])

2×3 Array{Int64,2}:
 1  1  1
 2  3  2

In [11]:
# test stuff

struct SquaresVector <: AbstractArray{Int,3}
    count::Int
end

Base.size(S::SquaresVector) = (S.count,S.count,S.count)

Base.getindex(S::SquaresVector, I::Vararg{Int,3}) = I[1]*I[2]*I[3]

In [12]:
SquaresVector(5)

5×5×5 SquaresVector:
[:, :, 1] =
 1   2   3   4   5
 2   4   6   8  10
 3   6   9  12  15
 4   8  12  16  20
 5  10  15  20  25

[:, :, 2] =
  2   4   6   8  10
  4   8  12  16  20
  6  12  18  24  30
  8  16  24  32  40
 10  20  30  40  50

[:, :, 3] =
  3   6   9  12  15
  6  12  18  24  30
  9  18  27  36  45
 12  24  36  48  60
 15  30  45  60  75

[:, :, 4] =
  4   8  12  16   20
  8  16  24  32   40
 12  24  36  48   60
 16  32  48  64   80
 20  40  60  80  100

[:, :, 5] =
  5  10  15   20   25
 10  20  30   40   50
 15  30  45   60   75
 20  40  60   80  100
 25  50  75  100  125

In [131]:
mat = [2 1 0;1 2 1;0 1 2]

3×3 Array{Int64,2}:
 2  1  0
 1  2  1
 0  1  2

In [132]:
b = [1 1 1]

1×3 Array{Int64,2}:
 1  1  1

In [133]:
mat/b

3×1 Array{Float64,2}:
 1.0000000000000002
 1.3333333333333335
 1.0000000000000002